In [99]:
import numpy as np
import scipy.signal as signal

# Chcel by som implementovať ešte max pool, avg pool, dense layer, sigmoid, cross-entropy

In [100]:

def print_matrix(arr):
    
    for i in range(arr.shape[0]):
        if len(arr.shape) > 1:
            for j in range(arr.shape[1]):
                print(" |", "{:>6.2f}".format(arr[i, j]), end="")
        else:
            print(" |", "{:>6.2f}".format(arr[i]), end="")
        print(" |")

In [101]:
def print_operation(arr1, operation, arr2, constant = None):
    if (arr1 is not None and (arr2 is None or arr1.shape[0] >= arr2.shape[0])):
        center = int(np.floor(arr1.shape[0] / 2))
        if (arr2 is not None):
            padding = int(np.ceil((arr1.shape[0] - arr2.shape[0]) / 2))
        
        for i in range(arr1.shape[0]):
            if constant is not None and i == center:
                print("{:>5}".format(constant), " * ", end="")
            
            if constant is not None and i != center:
                print(" " * 8, end="")
            
            if (len(arr1.shape) > 1):
                    
                for j in range(arr1.shape[1]):
                    print(" |", "{:>5.2f}".format(arr1[i, j]), end="")
            else:
                print(" |", "{:>5.2f}".format(arr1[i]), end="")
            
            if (arr2 is not None):
                if (i == center):
                    print(" |", end="")
                    print(" " * 3, end="")
                    print(operation, end="")
                    
                    if (i >= padding and i - padding < arr2.shape[0]):
                        print(" " * 3, end="")
                        if len(arr2.shape) > 1:
                            for j in range(arr2.shape[1]):
                                print(" |", "{:>5.2f}".format(arr2[i - padding, j]), end="")
                        else:
                            print(" |", "{:>5.2f}".format(arr2[i - padding]), end="")
                else:
                    if (i >= padding and i - padding < arr2.shape[0]):
                        print(" |", end="")
                        print(" " * 7, end="")
                        if len(arr2.shape) > 1:
                            for j in range(arr2.shape[1]):
                                print(" |", "{:>5.2f}".format(arr2[i - padding, j]), end="")
                        else:
                            print(" |", "{:>5.2f}".format(arr2[i - padding]), end="")
            print(" |")
    
    else:
        center = int(np.floor(arr2.shape[0] / 2))
        if (arr1 is not None):
            padding = int(np.ceil((arr2.shape[0] - arr1.shape[0]) / 2))
        
        for i in range(arr2.shape[0]):
            if constant is not None and i == center:
                print("{:>5}".format(constant), " * ", end="")
            
            if constant is not None and i != center:
                print(" " * 9, end="")
            
            if (arr1 is not None):
                if (i >= padding and i - padding < arr1.shape[0]):
                    if len(arr1.shape) > 1:
                        for j in range(arr1.shape[1]):
                            print(" |", "{:>5.2f}".format(arr1[i - padding, j]), end="")
                    else:
                        print(" |", "{:>5.2f}".format(arr1[i - padding]), end="")
                else:
                    for _ in range(arr1.shape[1]):
                        print(" " * 8, end="")
                    print("  ", end="")
                     
            if (i == center):
                if (arr1 is not None): 
                    print(" |", end="")
                    print(" " * 3, end="")
                    print(operation, end="")
                    print(" " * 3, end="")
                else:
                    print(" " * 7, end="")
                if len(arr2.shape) > 1:
                    for j in range(arr2.shape[1]):
                        print(" |", "{:>5.2f}".format(arr2[i, j]), end="")
                else:
                    print(" |", "{:>5.2f}".format(arr2[i]), end="")
            else:
                if (arr1 is not None):
                    if (i >= padding and i - padding < arr1.shape[0]):
                        print(" |", end="")
                print(" " * 7, end="")
                if len(arr2.shape) > 1:
                    for j in range(arr2.shape[1]):
                        print(" |", "{:>5.2f}".format(arr2[i, j]), end="")
                else:
                    print(" |", "{:>5.2f}".format(arr2[i]), end="")
            print(" |")
        
                

In [102]:
print_operation(None,
                "*",
                np.array([[11.111, 11.111, 11.111], [11.111, 1.111, 11.111], [11.111, 11.111, 11.111], [11.111, 11.111, 11.111]]), 0.01)

                 | 11.11 | 11.11 | 11.11 |
                 | 11.11 |  1.11 | 11.11 |
 0.01  *         | 11.11 | 11.11 | 11.11 |
                 | 11.11 | 11.11 | 11.11 |


In [103]:
class Conv:
    def __init__(self, input_shape, kernel_size, kernel_shape = (3, 3), stride = 1, padding = 0) -> None:
        input_depth, input_height, input_width = input_shape
        self.kernel_size = kernel_size
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.input_height = input_height
        self.input_width = input_width
        self.kernel = kernel_shape
        self.kernel_shape = (kernel_size, input_depth, *kernel_shape)
        self.output_shape = (kernel_size, int((input_height - kernel_shape[0] + 2 * padding) / stride + 1), int((input_width - kernel_shape[0] + 2 * padding) / stride + 1))
        self.biases = np.random.rand(*self.output_shape)
        self.kernels = np.random.rand(*self.kernel_shape)
        self.stride = stride
        self.zero_padding = padding
        
    
    def forward(self, input, verbose):
        if verbose: print(f'================================================')
        if verbose: print(f'Conv: Forward stride: {self.stride} padding: {self.zero_padding} kernel_size {self.kernel_size} kernel: {self.kernel}')
        if verbose: print(f'Skopírovanie biasov na výstup:')
        self.input = input
        self.output = np.copy(self.biases)
        
        for i in range(self.kernel_size):
            for j in range(self.input_depth):
                if verbose: print(f'Bias pre {i+1}. filter:')
                if verbose: print_matrix(self.output[i])
                if verbose: next_step()
                if (self.zero_padding > 0):
                    if verbose: print(f"Pridanie zero padding vstupnej matici")
                padded_input = np.pad(input[j], self.zero_padding)
                if (self.zero_padding > 0):
                    if verbose: print_matrix(padded_input)
                if verbose: print(f'Výpočet konvolúcii na {j+1}. matici:')
                if verbose: print_matrix(padded_input)
                if verbose: print(f'S {i+1}. filtrom:')
                if verbose: print_matrix(self.kernels[i, j])
                if verbose: next_step()
                windowed_input = np.lib.stride_tricks.sliding_window_view(padded_input, window_shape = self.kernel)[::self.stride, ::self.stride]
                for index_row, windowed_steps in enumerate(windowed_input):
                    for index_col, window in enumerate(windowed_steps):
                        if verbose: print(f"Prenásobenie okna filtrom:")
                        if verbose: print_operation(window, "*", self.kernels[i, j])
                        if verbose: next_step()
                        multiply = np.multiply(window, self.kernels[i, j])
                        if verbose: print(f"Vypočítanie a pričítanie konvolúcie k pozícii [{i}][{index_row}][{index_col}]:")
                        suma = np.sum(multiply)
                        if verbose: print_operation(multiply, "suma je:", np.array([[suma]]))
                        self.output[i][index_row][index_col] += suma
            if verbose: print(f"Výsledná matica pre {i+1}. filter je:")
            if verbose: print_matrix(self.output[i])
            if verbose: next_step()
                    
                        
        if verbose: print(f'================================================')
        if verbose: next_step()
        
        return self.output
    
    def backward(self, output_grads, learning_rate, verbose) -> None:
        if verbose: print(f'================================================')
        if verbose: print(f'Conv: Backpropagation stride: {self.stride} padding: {self.zero_padding} kernel_size {self.kernel_size} kernel: {self.kernel}')
        kernel_grads = np.zeros(self.kernels.shape)
        input_grads = np.zeros(self.input.shape)
        
        for i in range(self.kernel_size):
            padded_output_grands = np.pad(output_grads[i], self.kernel[0] - 1)
            windowed_output_grads = np.lib.stride_tricks.sliding_window_view(padded_output_grands, window_shape = self.kernel)[::self.stride, ::self.stride]
            if verbose: print(f'Gradient')
            if verbose: print_matrix(output_grads[i])
            if verbose: next_step()
            if verbose: print(f'Ohraničený gradient nulami pre jednoduchšie počítanie výstupných gradientov')
            if verbose: print_matrix(padded_output_grands)
            if verbose: next_step()
            for j in range(self.input_depth):
                padded_input = np.pad(self.input[j], self.zero_padding)
                windowed_input = np.lib.stride_tricks.sliding_window_view(padded_input, window_shape = output_grads[i].shape)[::self.stride + int((self.input_width - output_grads[i].shape[0] + 2 * self.zero_padding) / (self.kernel[0] - 1) - 1), ::self.stride + int((self.input_height - output_grads[i].shape[1] + 2 * self.zero_padding) / (self.kernel[1] - 1) - 1)]
                if verbose: print(f'Výpočet gradientov pre {i+1}. filter, gradient ako filter:')
                if verbose: print_matrix(output_grads[i])
                if verbose: next_step()
                if verbose: print(f'Bude prechádzať maticu:')
                if verbose: print_matrix(padded_input)
                if verbose: next_step()
                
                for index_row, windowed_steps in enumerate(windowed_input):
                    for index_col, window in enumerate(windowed_steps):
                        if verbose: print(f'Výpočet gradientu pre {i+1} filter na pozícii [{i}, {j}][{index_row}][{index_col}]')
                        if verbose: print_operation(window, "*", output_grads[i])
                        if verbose: next_step()
                        if verbose: print(f'Výsledok násobenia:')
                        multiply = np.multiply(window, output_grads[i])
                        if verbose: print_matrix(multiply)
                        if verbose: next_step()
                        if verbose: print(f'Sčítanie:')
                        sum = np.sum(multiply)
                        if verbose: print_operation(multiply, "suma je:", np.array([[sum]]))
                        if verbose: next_step()
                        kernel_grads[i, j][index_row][index_col] = sum
                        
                if verbose: print(f'Výsledný gradient pre {i + 1} filter')
                if verbose: print_matrix(kernel_grads[i, j])
                if verbose: next_step()
                
                if verbose: print(f'Výpočet výstupných gradientov')
                for index_row, windowed_steps in enumerate(windowed_output_grads):
                    for index_col, window in enumerate(windowed_steps):
                        if verbose: print(f'Výpočet výstupného gradientu na pozíciu [{j}][{index_row}][{index_col}]')
                        if verbose: print(f'Násobanie okna filtrom:')
                        if verbose: print_operation(window, "*", self.kernels[i, j])
                        if verbose: next_step()
                        multiply = np.multiply(window, self.kernels[i, j])
                        if verbose: print(f'Suma a pričítanie na pozíciu [{j}][{index_row}][{index_col}]')
                        sum = np.sum(multiply)
                        if verbose: print_operation(multiply, "suma je:", np.array([[sum]]))
                        if verbose: next_step()
                        input_grads[j][index_row][index_col] += sum
                if verbose: print(f'Výstupné gradienty pre {j} dimenziu')
                if verbose: print_matrix(input_grads[j])
                if verbose: next_step()
        
        for j in range(self.input_depth):
            input_grads[j] = np.rot90(input_grads[j], 2)
        
        if verbose: print(f'Výpočet nových filtrov.')
        for i in range(self.kernel_size):
            for j in range(self.input_depth):
                if verbose: print(f'Výpočet {i+1}. filtru')
                if verbose: print_operation(kernel_grads[i, j], "*", None, learning_rate)
                if verbose: print(f'Výsledok výpočtu {i+1}. filtru')
                self.kernels[i, j] -= learning_rate * kernel_grads[i, j]
                if verbose: print_matrix(self.kernels[i, j])
                if verbose: next_step()
        
        if verbose: print(f'Výpočet nových biasov.')
        for i in range(self.kernel_size):
            if verbose: print(f'Výpočet {i+1}. biasu')
            if verbose: print_operation(output_grads[i], "*", None, learning_rate)
            if verbose: print(f'Výsledok výpočtu {i+1}. biasu')
            self.biases[i] -= learning_rate * output_grads[i]
            if verbose: print_matrix(self.biases[i])
            if verbose: next_step()
        
        if verbose: print(f'================================================')
        if verbose: next_step()
        
        return input_grads
        

In [104]:
class MaxPool():
    def __init__(self, input_shape, depth):
        input_depth, input_height, input_width = input_shape
        self.input_shape = input_shape
        self.depth = depth
        self.input_depth = input_depth
        self.input_height = input_height
        self.input_width = input_width
        self.output_shape = (input_depth, int(input_height / depth), int(input_width / depth))
        self.kernel = (depth, depth)
        self.indices = np.zeros(self.output_shape)
    
    def forward(self, input, verbose):
        self.input = input
        if verbose: print(f'================================================')
        if verbose: print(f'MaxPool: Forward (2, 2)')
        
        help_indices = np.arange(0, self.input_height * self.input_width).reshape((self.input_height, self.input_width))
        output = np.zeros(self.output_shape)
        
        for i in range(self.input_depth):
            if verbose: print(f'Počítanie maxima pre jednotlivé okna vstupnej matice:')
            if verbose: print_matrix(input[i])
            if verbose: next_step()
            windowed_input = np.lib.stride_tricks.sliding_window_view(input[i], window_shape = self.kernel)[::2, ::2]
            windowed_indices = np.lib.stride_tricks.sliding_window_view(help_indices, window_shape = self.kernel)[::2, ::2]
            for index_row, (w_s, w_i_s) in enumerate(zip(windowed_input, windowed_indices)):
                for index_col, (window, idices_step) in enumerate(zip(w_s, w_i_s)):
                    if verbose: print(f'Počítanie maxima pre pozíciu [{i}][{index_row}][{index_col}]:')
                    maximum = window.max()
                    if verbose: print_operation(window, "maximum je: ", np.array([[maximum]]))
                    output[i][index_row][index_col] = maximum
                    self.indices[i][index_row][index_col] = idices_step.reshape(-1)[window.argmax()]
                    if verbose: next_step()
            if verbose: print(f'Výstupná matica maxím:')
            if verbose: print_matrix(output[i])
            if verbose: next_step()
                    
        if verbose: print(f'================================================')
        if verbose: next_step()
        return output
            
        
    
    def backward(self, output_grads, learning_rate, verbose):
        if verbose: print(f'================================================')
        if verbose: print(f'MaxPool: Backpropagation (2, 2)')
        if verbose: print(f'Spropagovanie gradientov na pozície maxím vstupného vektoru')
        input_grads = np.zeros(self.input_shape).reshape(self.input_depth, -1)
        indices = self.indices.reshape(self.input_depth, -1).astype(int)
        grads = output_grads.reshape(self.input_depth, -1)
        for i in range(self.input_depth):
            if verbose: print(f'Gradient pre {i+1}. maticu: ')
            if verbose: print_matrix(output_grads[i])
            if verbose: print(f'Pozície maxím pre {i+1}. maticu:')
            if verbose: print_matrix(self.indices[i])
            if verbose: print(f'Propagácia pre {i+1}. maticu:')
            for j, k in enumerate(indices[i]):
                input_grads[i][k] += grads[i][j]
            if verbose: print_matrix(input_grads[i].reshape((self.input_shape[1], self.input_shape[2])))
        input_grads = input_grads.reshape(self.input_shape)
        if verbose: print(f'================================================')
        if verbose: next_step()
        return input_grads
            
        

In [105]:

class AvgPool():
    def __init__(self, input_shape, depth):
        input_depth, input_height, input_width = input_shape
        self.input_shape = input_shape
        self.depth = depth
        self.input_depth = input_depth
        self.input_height = input_height
        self.input_width = input_width
        self.output_shape = (input_depth, int(input_height / depth), int(input_width / depth))
        self.kernel = (depth, depth)
    
    def forward(self, input, verbose):
        output = np.zeros(self.output_shape)
        
        if verbose: print(f'================================================')
        if verbose: print(f'Forward: AvgPool (2, 2)')
        
        for i in range(self.input_depth):
            if verbose: print(f'Počítanie priemeru pre jednotlivé okná vstupnej matice:')
            if verbose: print_matrix(input[i])
            if verbose: next_step()
            windowed_input = np.lib.stride_tricks.sliding_window_view(input[i], window_shape = self.kernel)[::2, ::2]
            for index_row, w_s in enumerate(windowed_input):
                for index_col, window in enumerate(w_s):
                    if verbose: print(f'Výpočet primeru pre pozíciu [{i}][{index_row}][{index_col}]:')
                    average = np.average(window)
                    if verbose: print_operation(window, "priemer je:", np.array([[average]]))
                    output[i][index_row][index_col] = average
                    if verbose: next_step()
            if verbose: print(f'Výstupná matica priemerov:')
            if verbose: print_matrix(output[i])
            if verbose: next_step()
        if verbose: print(f'================================================')
        if verbose: next_step()
        return output
    
    def backward(self, output_grads, learning_rate, verbose):
        input_grads = np.ones(self.input_shape)
        
        if verbose: print(f'================================================')
        if verbose: print(f'Backpropagation: AvgPool (2, 2)')
        if verbose: print(f'Priemerné rozdelenie gradientu medzi spriemerované prvky okien:')
        
        for i in range(self.input_depth):
            if verbose: print(f'{i+1}. matica gradientov:')
            if verbose: print_matrix(output_grads[i])
            if verbose: next_step()
            if verbose: print(f'Expanzia hodnôt gradientu na veľkosť filtra:')
            expanded_output_grads = np.kron(output_grads[i], np.ones(self.kernel))
            if verbose: print_matrix(expanded_output_grads)
            if verbose: next_step()
            if verbose: print(f'Pridanie hodnôť gradientu do výslednej matice:')
            input_grads[i][0:expanded_output_grads.shape[0], 0:expanded_output_grads.shape[1]] = expanded_output_grads
            if verbose: print_matrix(input_grads[i])
            if verbose: next_step()
            if verbose: print(f'Spriemerovanie matice podľa velkosti filtra:')
            input_grads[i] = input_grads[i] * (1 / (self.output_shape[1] * self.output_shape[2]))
            if verbose: print_matrix(input_grads[i])
            if verbose: next_step()
                    
        if verbose: print(f'================================================')
        if verbose: next_step()
        return input_grads
                
        

In [106]:
class Sigmoid():
    def __sigmoid(self, input):
        return 1 / (1 + np.exp(np.negative(input)))
    
    def forward(self, input, verbose):
        self.input = input
        if verbose: print(f'================================================')
        if verbose: print(f'Forward: sigmoid')
        if verbose: print(f'Použitie funkcie sigmoidy na vstupnú maticu:')
        if (len(input.shape) > 2):
            if verbose: print_matrix(input[0])
        else:
            if verbose: print_matrix(input)
        if verbose: next_step()
        if verbose: print(f'Výsledok')
        sigmoid_input = self.__sigmoid(input)
        if (len(sigmoid_input.shape) > 2):
            if verbose: print_matrix(sigmoid_input[0])
        else:
            if verbose: print_matrix(sigmoid_input)
        if verbose: print(f'================================================')
        if verbose: next_step()
        return sigmoid_input
    
    def backward(self, output_grads, learning_rate, verbose):
        if verbose: print(f'================================================')
        if verbose: print(f'Backward: sigmoid')
        if verbose: print(f'Použitie funkcie sigmoidy na vstupnú maticu:')
        if (len(self.input.shape) > 2):
            if verbose: print_matrix(self.input[0])
        else:
            if verbose: print_matrix(self.input)
        if verbose: print(f'Výsledok')
        s = self.__sigmoid(self.input)
        if verbose: next_step()
        if (len(s.shape) > 2):
            if verbose: print_matrix(s[0])
        else:
            if verbose: print_matrix(s)
        if verbose: print(f'Vynásovanie matice gradientov s deriváciou sigmoidy:')
        if (len(output_grads.shape) > 2 and len((s * (1 - s)).shape) > 2):
            if verbose: print_operation(output_grads[0], "*", (s * (1 - s))[0])
        else:
            if verbose: print_operation(output_grads, "*", (s * (1 - s)))
        if verbose: next_step()
        input_grads = np.multiply(output_grads, (s * (1 - s)))
        if verbose: print(f'Výsledok')
        if (len(input_grads.shape) > 2):
            if verbose: print_matrix(input_grads[0])
        else:
            if verbose: print_matrix(input_grads)
        if verbose: print(f'================================================')
        if verbose: next_step()
        return input_grads

In [107]:
class Reshape():
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape
    
    def forward(self, input, verbose):
        if verbose: print(f'================================================')
        if verbose: print(f'Forward: reshape {self.input_shape}, {self.output_shape}')
        if verbose: print(f'Zmena tvaru matice')
        if verbose: next_step()
        if (len(input.shape) > 2):
            if verbose: print_matrix(input[0])
        else:
            if verbose: print_matrix(input)
        if verbose: print(f'Výsledok')
        reshaped_input = input.reshape(self.output_shape)
        if verbose: print_matrix(reshaped_input)
        if verbose: print(f'================================================')
        if verbose: next_step()
        return reshaped_input
    
    def backward(self, output_grad, learning_rate, verbose):
        if verbose: print(f'================================================')
        if verbose: print(f'Backward: reshape {self.input_shape}, {self.output_shape}')
        if verbose: print(f'Zmena tvaru matice')
        if verbose: print_matrix(output_grad)
        reshaped_input = output_grad.reshape(self.input_shape)
        if verbose: next_step()
        if verbose: print(f'Výsledok')
        if (len(reshaped_input.shape) > 2):
            if verbose: print_matrix(reshaped_input[0])
        else:
            if verbose: print_matrix(reshaped_input)
        if verbose: print(f'================================================')
        if verbose: next_step()
        return reshaped_input

In [108]:
class Dense():
    def __init__(self, input_shape, output_shape):
        self.weights = np.random.randn(output_shape, input_shape)
        self.biases = np.random.randn(output_shape, 1)
        self.input_shape = input_shape
        self.output_shape = output_shape
        
    
    def forward(self, input, verbose):
        self.input = input
        if verbose: print(f'================================================')
        if verbose: print(f'Forward: Dense z {self.input_shape} do {self.output_shape}')
        if verbose: next_step()
        if verbose: print(f'Násobenie matice váh s maticou vstupu a pričítanie biasu')
        if verbose: print_operation(self.weights, "*", input)
        weight_input = np.dot(self.weights, input)
        if verbose: next_step()
        if verbose: print(f'Výsledok:')
        if verbose: print_matrix(weight_input)
        if verbose: next_step()
        if verbose: print(f'Pričítanie biasu (ku každému prvku z matice váh sa pričíta každý prvok z matice biasov):')
        if verbose: print_operation(weight_input, "+", self.biases)
        weight_input_biases = weight_input + self.biases
        if verbose: next_step()
        if verbose: print(f'Výsledok:')
        if verbose: print_matrix(weight_input_biases)
        if verbose: next_step()
        if verbose: print(f'================================================')
        
        
        return weight_input_biases
    
    def backward(self, output_grads, learning_rate, verbose):
        if verbose: print(f'================================================')
        if verbose: print(f'Backpropagation: Dense z {self.output_shape} do {self.input_shape}')
        if verbose: print(f'Násobenie gradientov so vstupom:')
        if verbose: next_step()
        if verbose: print_operation(output_grads, "*" , self.input.T)
        output_grads_input = np.dot(output_grads, self.input.T)
        if verbose: next_step()
        if verbose: print(f'Výsledok:')
        if verbose: print_matrix(output_grads_input)
        if verbose: next_step()
        if verbose: print(f'Násobenie vynásobených gradientov so vstupom s learning rate:')
        if verbose: print_operation(output_grads_input, "*", None, learning_rate)
        if verbose: next_step()
        if verbose: print(f'Výsledok:')
        learning_output_grads_input = learning_rate * output_grads_input
        if verbose: print_matrix(learning_output_grads_input)
        if verbose: next_step()
        if verbose: print(f'Odčítanie stávajúcich váh s prechozím výsledkom:')
        if verbose: print_operation(self.weights, "-", learning_output_grads_input)
        if verbose: next_step()
        if verbose: print(f'Výsledok:')
        self.weights -= learning_output_grads_input
        if verbose: print_matrix(self.weights)
        if verbose: next_step()
        if verbose: print(f'Vynásobenie learning rate s gradientmi:')
        learning_grads = learning_rate * output_grads
        if verbose: print_operation(output_grads, "*", None, learning_rate)
        if verbose: next_step()
        if verbose: print(f'Výsledok:')
        if verbose: print_matrix(learning_grads)
        if verbose: next_step()
        if verbose: print(f'Odčítanie výsledku od stávajúcich biaseov:')
        if verbose: print_operation(self.biases, "-", learning_grads)
        if verbose: next_step()
        if verbose: print(f'Výsledok:')
        if verbose: print_matrix(self.biases)
        self.biases -= learning_rate * output_grads
        
        return np.dot(self.weights.T, output_grads)
        
        

In [109]:
def next_step():
    input("Ďalší krok [stlačte ľubovolné tlačítko]")

In [110]:
def binary_cross_entropy(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

In [127]:
def predict(network, input, verbose):
    output = input
    for layer in network:
        output = layer.forward(output, verbose)
    return output

def train(network, loss, loss_prime, x_train, y_train, epochs = 1000, learning_rate = 0.01, print_calculations = True, verbose = True):
    for e in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # forward
            output = predict(network, x, print_calculations)

            # error
            error += loss(y, output)

            # backward
            grad = loss_prime(y, output)
            for i, layer in enumerate(reversed(network)):
                try:
                    grad = layer.backward(grad, learning_rate, print_calculations)
                except:
                    raise

        error /= len(x_train)
        if verbose:
            if verbose: print(f"{e + 1}/{epochs}, error={error}")

In [148]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils

x_train = np.array([[[[1, -1, 1, 1, -1, 1], [1, -1, 1, 1, 1, -1], [1, 1, -1, 1, -1, 1], [-1, 1, 1, -1, 1, -1], [1, -1, 1, -1, 1, -1], [1, -1, 1, -1, 1, -1]]]])
y_train = np.array([[[0], [1]]])

# neural network
network = [
    Conv((1, 6, 6), 2, (3, 3), 2, 1),
    Sigmoid(),
    MaxPool((2, 3, 3), 2),
    Reshape((2, 1, 1), (2 * 1 * 1, 1)),
    Dense(2 * 1 * 1, 2),
    Sigmoid()
]

# train
train(
    network,
    binary_cross_entropy,
    binary_cross_entropy_prime,
    x_train,
    y_train,
    epochs=10,
    learning_rate=0.1,
    print_calculations=True,
)

predict(network, x_train[0], False)

Conv: Forward stride: 2 padding: 1 kernel_size 2 kernel: (3, 3)
Skopírovanie biasov na výstup:
Bias pre 1. filter:
 |   0.10 |   0.78 |   0.67 |
 |   0.67 |   0.15 |   0.31 |
 |   0.94 |   0.10 |   0.16 |
Pridanie zero padding vstupnej matici
 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |
 |   0.00 |   1.00 |  -1.00 |   1.00 |   1.00 |  -1.00 |   1.00 |   0.00 |
 |   0.00 |   1.00 |  -1.00 |   1.00 |   1.00 |   1.00 |  -1.00 |   0.00 |
 |   0.00 |   1.00 |   1.00 |  -1.00 |   1.00 |  -1.00 |   1.00 |   0.00 |
 |   0.00 |  -1.00 |   1.00 |   1.00 |  -1.00 |   1.00 |  -1.00 |   0.00 |
 |   0.00 |   1.00 |  -1.00 |   1.00 |  -1.00 |   1.00 |  -1.00 |   0.00 |
 |   0.00 |   1.00 |  -1.00 |   1.00 |  -1.00 |   1.00 |  -1.00 |   0.00 |
 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |
Výpočet konvolúcii na 1. matici:
 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |
 |   0.00 |   1.00 |  -1.00 |   1.00 |   1.00 |  

KeyboardInterrupt: Interrupted by user